In [1]:
from bs4 import BeautifulSoup
import urllib
import re

import urllib2
import json

In [2]:
#general format is https://www.kickstarter.com/projects/creatorname/campaignname

#Creating a function to grab detailed text information about the project from the project's kicstarter page
#Passing project id, url, and currency of funds requested from the meta data of successful project URLs
# URLS from './data/project_successful_meta.json' created by the project_metadata.ipyth program
def get_one_project_det(project_det):
    
    currency = project_det["currency"]
    url =  project_det["url"]
    
    #Creating beautiful soup object from URL
    r = urllib.urlopen(url)
    soup = BeautifulSoup(r)
    
    
    #Need to obtain the donation values and the reward/gifts associated with each donation value
    #Reading in tag with multiple pieces of info on this
    b=list(soup.find_all(class_="NS_projects__rewards_list js-project-rewards"))
    #Donation dollar amount group
    reward_value_list = [reward.string.split()[0] for reward in b[0]("h5",class_="mb1")]
    #Number of backers associated with Donation dollar amount group
    backer_list = [backers.string.split()[0]+' backers' for backers in b[0]("span", class_="num-backers mr1")]
    #Reward/gift associated with Donation dollar amount group
    reward_item_list = [item("p")[0].get_text() for item in b[0]("div",class_="desc h5 mb2 break-word")]
    
    #removing currency sign from reward value
    reg = re.compile("\d.*",flags=re.U)
    reward_value_list = [reg.findall(reward.string.split()[0])[0] for reward in b[0]("h5",class_="mb1")]

    #Text associated with the risks of the project--outlining project hurdles
    c=list(soup("div", class_="mb6 mb2"))
    risk=[item("p") for item in c][0]
    
    #Creating a dictionary for each project id 
    #project_det = dict()
    #Project title 
    project_det["title"] = soup.find_all("meta", property="og:title")[0].get("content")
    #Blurb about project 
    #project_det["blurb1"] = soup.find_all(property=re.compile("og:description"))[0].get("content")
    #Creator's URL hompage
    project_det["creator_url"] = soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
    #Total number of backers
    project_det["no_backers"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[1].string
    #Project goal amount---min value needed to be raised for successful project 
    project_det["goal"] = reg.findall(list(soup.find_all("span", class_="money " +currency.lower()+ " no-code")[2])[0].string)
    #Number of dollars raised during campaign 
    project_det["no_dollars_raised"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[3].string
    #Information about donation amount groups, number of backer for group and gift associated with group 
    project_det["reward_backer_tup"] = (zip(reward_value_list, backer_list,reward_item_list))
    #Creator's detailed information about risks involved with the project 
    project_det["risk"] = [para.get_text() for para in risk]
    #project id 
    #project_det["id"] = pid
    #project url 
    #project_det["url"] = url
    
    return(project_det)

In [3]:
#open url information
fd = open('./data/project_successful_meta.json', 'r')
text = fd.read()
fd.close()

pmeta = json.loads(text)
len(pmeta)

2729

In [4]:
#call function created above to grab project details and loop through project for every url in pmeta file
# for homework purposes, limited this to 500 urls, but we'll pull more for final project
project_succ_det = []
for ind, project_header in enumerate(pmeta[2000:]):
    try:
        print("trying index", ind)
        project_succ_det.append(get_one_project_det(project_header))
    except:
        print("errored out for index", ind)

('trying index', 0)
('trying index', 1)
('trying index', 2)
('trying index', 3)
('trying index', 4)
('trying index', 5)
('trying index', 6)
('trying index', 7)
('trying index', 8)
('trying index', 9)
('trying index', 10)
('trying index', 11)
('trying index', 12)
('trying index', 13)
('trying index', 14)
('trying index', 15)
('trying index', 16)
('trying index', 17)
('trying index', 18)
('trying index', 19)
('trying index', 20)
('trying index', 21)
('trying index', 22)
('trying index', 23)
('trying index', 24)
('trying index', 25)
('trying index', 26)
('trying index', 27)
('trying index', 28)
('trying index', 29)
('trying index', 30)
('trying index', 31)
('trying index', 32)
('trying index', 33)
('trying index', 34)
('trying index', 35)
('trying index', 36)
('trying index', 37)
('trying index', 38)
('trying index', 39)
('trying index', 40)
('trying index', 41)
('trying index', 42)
('trying index', 43)
('trying index', 44)
('trying index', 45)
('trying index', 46)
('trying index', 47)
('

In [ ]:
project_header.keys()

In [5]:
len(project_succ_det)

278

In [6]:

#Save data to file for further use
project_meta_json = json.dumps(obj = project_succ_det, indent = 4)
#File to look at for sample data
fd = open("./data/project_successful_detail.json", "a")
fd.write(project_meta_json)
fd.close()